In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Tito Osadebey\\Desktop\\9ja_food_class\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\Tito Osadebey\\Desktop\\9ja_food_class'

In [9]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [11]:
from src.ngaFoodClassifier.constants import *
from src.ngaFoodClassifier.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH):
        #params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        # self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config
      

In [13]:
import os
import urllib.request as request
import zipfile
from src.ngaFoodClassifier import logger
from src.ngaFoodClassifier.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logger.info(f"File already exists with size: {get_size(Path(self.config.local_data_file))}")   

    
    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [15]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-30 17:36:38,988: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-06-30 17:36:39,512: INFO: common: created directory at: artifacts]
[2024-06-30 17:36:39,515: INFO: common: created directory at: artifacts/data_ingestion]
[2024-06-30 17:36:59,549: INFO: 1113320345: artifacts/data_ingestion/data.zip downloaded! with following info: 
Connection: close
Content-Length: 15755649
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "6975a247f9918bb46c79004b29ab735da55638e7e43c16f5fae9062ee60382a4"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 49B7:36F1BB:2445027:2643AE5:668181A8
Accept-Ranges: bytes
Date: Sun, 30 Jun 2024 16:36:39 GMT
Via: 1.1 varnish
X-Served-By: cache-mrs1050093-MRS
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S1719765399.487107,VS0,VE141
Vary: Authorization,